# Решение #4 - Кластеризация

### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

import warnings
warnings.filterwarnings("ignore")

df = pd.read_json('./../data/sample.json')

### Предобработка строк

In [2]:
df['text_preprocess'] = df['text'].apply(load_dataset.preprocess_text)
strings = list(df['text_preprocess'])

In [3]:
strings

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей пбмолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал влера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

### Структура датасета

In [4]:
df.head()

,id,text,text_preprocess
0,1,Ты нашёл их или нет?,ты нашел их или нет
1,2,Почему она так со мной поступает?,почему она так со мной поступает
2,3,Никто туда больше не ходит.,никто туда больше не ходит
3,4,У него с собой не было тогда денег.,у него с собой не было тогда денег
4,5,Почему они с нами так поступают?,почему они с нами так поступают


In [103]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import math
from typing import List

# np.seterr(divide='ignore', invalid='ignore') # игнорирование предупреждений

def make_cluster(strings:List[str]):
    if len(set(strings)) == 1:
        return [strings]
    # Векторизация
    Vectorizer = CountVectorizer()
    vector = Vectorizer.fit_transform(strings)
    strings_vectorized = vector.toarray()

    # Уменьшение размерности
    pca_model = PCA(n_components=2).fit(strings_vectorized)
    strings_vectorized_2n = pca_model.transform(strings_vectorized)

    # Кластеризация
    diff_x = max(strings_vectorized_2n[:,0]) - min(strings_vectorized_2n[:,0])
    diff_y = max(strings_vectorized_2n[:,1]) - min(strings_vectorized_2n[:,1])

    eps = (diff_x * diff_y)**0.6 / math.log(len(strings),2)
    if eps == 0.0:
        return [strings]
    # print('eps = ',eps)
    model = DBSCAN(eps=eps,min_samples=1).fit(strings_vectorized_2n)
    clusters = {}
    for i in range(len(model.labels_)):
        if model.labels_[i] not in clusters:
            clusters[model.labels_[i]] = []
        clusters[model.labels_[i]].append(strings[i])

    # plt.scatter(strings_vectorized_2n[:,0],strings_vectorized_2n[:,1],c=model.labels_)
    return list(clusters.values())

In [107]:
clusters = make_cluster(strings)
for iter in range(10):
    for i,cluster in enumerate(clusters):
        clusters[i] = make_cluster(cluster);
    clusters = [item for cluster in clusters for item in cluster];

In [108]:
[sorted(cluster) for cluster in clusters]

[['ты нашел их или нет'],
 ['он всю ночь стонал от сильной боли'],
 ['говорят в горах погода быстро меняется',
  'говорят погода в горах быстро меняется',
  'зачем тому была нужна отвертка',
  'здесь есть поблизости банкомат',
  'здесь поблизости есть банкомат',
  'им были нужны деньги',
  'им нужны были деньги',
  'мы должны всем им помочь',
  'мы много времени провгдим вместе',
  'мы много времени проводим вместе',
  'мы сегодня поговорим с томом',
  'мы теряем здесь время',
  'надо было позвонить и заранее забронировать столик',
  'он был тогда в америке',
  'он тогда был в америке',
  'она была тогда маленькой девочкой',
  'они пили белое сухое вино',
  'прибавить пять к десяти легко',
  'рому было нечего сказать',
  'сколько сейчас времени в париже',
  'тому был нужен молоток',
  'тому было нечего сказать',
  'тому некому было помочь',
  'тому нужен был молоток',
  'у тебя в детстве была любимая книга',
  'я буду там в пять',
  'я был вынужден уйти',
  'я вам покажу несколько фото

In [109]:
[len(cluster) for cluster in clusters]

[1,
 1,
 48,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 8,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 4,
 1,
 2,
 1,
 2,
 2,
 1,
 4,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 3,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,

In [192]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

Vectorizer = CountVectorizer()
vector = Vectorizer.fit_transform(strings)
strings_vectorized = vector.toarray()
strings_vectorized_n = PCA(n_components=10).fit_transform(strings_vectorized)
model = KMeans(n_clusters=2).fit(strings_vectorized_n)
model.labels_

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,

# 2 Вариант

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import math
from typing import List



def km(strings):
    n_clusters = 2
    if len(set(strings)) <= n_clusters:
        return [strings]
        
    # Векторизация
    Vectorizer = CountVectorizer()
    vector = Vectorizer.fit_transform(strings)
    strings_vectorized = vector.toarray()

    # Уменьшение размерности
    strings_vectorized_n = PCA(n_components=len(strings_vectorized)//2).fit_transform(strings_vectorized)

    model = KMeans(n_clusters=n_clusters).fit(strings_vectorized_n)
    clusters = {}
    for i in range(len(model.labels_)):
        if model.labels_[i] not in clusters:
            clusters[model.labels_[i]] = []
        clusters[model.labels_[i]].append(strings[i])

    # plt.scatter(strings_vectorized_2n[:,0],strings_vectorized_2n[:,1],c=model.labels_)
    # plt.show()
    return list(clusters.values())

In [68]:
clusters = km(strings)
for iter in tqdm(range(len(strings)//8)):
    for i,cluster in enumerate(clusters):
        clusters[i] = km(cluster)
    clusters = [item for cluster in clusters for item in cluster];

  0%|          | 0/51 [00:00<?, ?it/s]

In [56]:
groups = []
threshold = 0.89
for cluster in tqdm(clusters):
    if len(cluster) == 1:
        groups.append(cluster)
    if len(cluster) == 2:
        if karlovskiy_distance.is_rewrite_karlovskiy_distance(cluster[0],cluster[1],threshold):
            groups.append(cluster)  
        else:
            # print([cluster[0]])
            # print([cluster[1]])
            groups.append([cluster[0]])
            groups.append([cluster[1]])
    if len(cluster) > 2:
        used = [False] * len(cluster)
        for i in range(len(cluster)):
            if used[i]:
                continue
            for j in range(i+1,len(cluster)):
                if used[j]:
                    continue
                if karlovskiy_distance.is_rewrite_karlovskiy_distance(cluster[i],cluster[j],threshold):
                    used[i] = True
                    used[j] = True
                    groups.append([cluster[i],cluster[j]])
        for i in range(len(cluster)):
            if not used[i]:
                groups.append([cluster[i]])

  0%|          | 0/256 [00:00<?, ?it/s]

In [66]:
len(groups)

283

In [64]:
groups

[['ты нашел их или нет'],
 ['ты играешь в футбол или регби', 'ты в футбол играешь или в регби'],
 ['ты любишь яблоки или апельсины', 'ты яблоки любишь или апельсины'],
 ['он всю ночь стонал от сильной боли'],
 ['я просто был невнимателен'],
 ['я был вынужден уйти'],
 ['тому был нужен молоток', 'тому нужен был молоток'],
 ['кен был вчера дома', 'кен вчера был дома'],
 ['он был тогда в америке', 'он тогда был в америке'],
 ['она была тогда маленькой девочкой'],
 ['тому было тогда всего тринадцать лет'],
 ['сколько тебе тогда было лет', 'сколько шебе тогда было лет'],
 ['тому было нечего сказать', 'рому было нечего сказать'],
 ['тому некому было помочь'],
 ['мост очень длинный и высокий', 'мост очень длинный и очень высокий'],
 ['эта способность к общению очень нам помогает',
  'эта способность к общению нам очень помогает'],
 ['я написал влера письмо'],
 ['я здесь часто ем'],
 ['я полностью с ними согласен'],
 ['прибавить пять к десяти легко'],
 ['я поймал сегодня три рыбы'],
 ['я тоже з

In [62]:
sorted([group[0] for group in groups if len(group) == 1])

['больше мы для него ничего не можем сделать',
 'вот как оно все обернулось',
 'где был том весь день',
 'думаю я мог бы и один это сделать',
 'если у тебя будет время завтра приходи ко мне',
 'зачем тому была нужна отвертка',
 'здесь ничего никогда не происходит',
 'иди сюда я тебе еще кое-что покажу',
 'интерфейс очень интуитивно понятен',
 'как бы ты посоветовал мне поступить',
 'какая гора самая высокая в мире',
 'когда я его видел в последний раз',
 'кто-то это уже сделал',
 'математик такого бы не сказал',
 'мне было не с кем поговорить',
 'мне никогда не нужно было столько денег',
 'мне сегодня не нужно никуда идти',
 'многие птицы на зиму улетают на юг',
 'многие птицы осенью улетают на юг',
 'можешь меня высадить у библиотеки',
 'мы больше не хотим ждать',
 'мы будем делать все по-своему',
 'мы должны всем им помочь',
 'мы их никогда не найдем',
 'мы не были так заняты',
 'мы сегодня поговорим с томом',
 'мы теряем здесь время',
 'мэри говорит что ничего не будет делать',
 'на

In [59]:
[group for group in groups if len(group) == 2]

[['ты играешь в футбол или регби', 'ты в футбол играешь или в регби'],
 ['ты любишь яблоки или апельсины', 'ты яблоки любишь или апельсины'],
 ['тому был нужен молоток', 'тому нужен был молоток'],
 ['кен был вчера дома', 'кен вчера был дома'],
 ['он был тогда в америке', 'он тогда был в америке'],
 ['сколько тебе тогда было лет', 'сколько шебе тогда было лет'],
 ['тому было нечего сказать', 'рому было нечего сказать'],
 ['мост очень длинный и высокий', 'мост очень длинный и очень высокий'],
 ['эта способность к общению очень нам помогает',
  'эта способность к общению нам очень помогает'],
 ['я просто хочу исчезнуть', 'я просто хоку исчезнуть'],
 ['я живу сейчас в бостоне', 'я сейчас живу в бостоне'],
 ['я вчера неплохо провел время', 'я неплохо провел вчера время'],
 ['мы много времени провгдим вместе', 'мы много времени проводим вместе'],
 ['я покажу вам несколько фотографий', 'я вам покажу несколько фотографий'],
 ['здесь есть поблизости банкомат', 'здесь поблизости есть банкомат'],

In [60]:
[group for group in groups if len(group) > 2]

[]

In [61]:
save_groups.save_groups(groups,"../output/solution4-result.json")

### Решение основано кластеризации методом К-средних
Применяются следующие алгоритмы:
* Растояние Карловского
* Метод К-средних

In [5]:
pd.options.mode.chained_assignment = None # Убираем лишние предупреждения
df['used'] = [False] * len(df)

# Функция фильтрации на основе алгоритма верификации рерайтов (function) и заданного ограничения (threshold) 
# c возможностью проверки наборов местоимений (check_pronoun) 
def filter(groups,function,threshold,check_pronoun=True):
    for i in tqdm(range(len(df))):
        if df['used'][i]:  # Проверка метки использования строки
            continue
        groups.append([])
        groups[-1].append(df['text'][i])
        df['used'][i] = True
        
        for j in range(i+1,len(df)):
            if df['used'][j]: # Проверка метки использования строки
                continue
            if check_pronoun:
                if(function(df['text_preprocess'][i],df['text_preprocess'][j],threshold) and pronouns.check_pronoun_correspondence(df['text_preprocess'][i],df['text_preprocess'][j])):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
            else:
                if(function(df['text_preprocess'][i],df['text_preprocess'][j],threshold)):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
        if len(groups[-1]) == 1: # Если строка не имеет рерайта, то группа не формируется
            df['used'][i] = False 
            groups = groups[:-1]
    return groups

groups = []
# Фильтрация на основе алгоритмов верификации рерайтов
groups = filter(groups,karlovskiy_distance.is_rewrite_karlovskiy_distance,0.89,check_pronoun=True)
groups = filter(groups,word_set.is_rewrite_word_set,threshold=0.99,check_pronoun=False)
groups = filter(groups,hamming_distance_custom.is_rewrite_hamming_distance_custom,threshold=0.9,check_pronoun=False)
groups = filter(groups,cosine_tfidf.is_rewrite_cosine_tfidf,0.7,check_pronoun=True)
groups = filter(groups,jaro_winkler.is_rewrite_jaro_winkler,0.9,check_pronoun=True)

# Добавляем уникальные строки (не имеющие рерайт)
groups = groups + [[str] for str in list(df[df['used']==False]['text'])]

df = df.drop('used',axis=1)

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

### Количество найденных групп

In [6]:
len(groups)

277

### Все найденные группы

In [7]:
groups

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Строки, которые имеют более 1 рерайта

In [8]:
groups3 = [group for group in groups if len(group) > 2]

In [9]:
len(groups3)

1

In [10]:
groups3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

### Строки, которые имеют ровно 1 рерайт

In [11]:
groups2 = [group for group in groups if len(group) == 2]

In [12]:
len(groups2)

133

In [13]:
groups2

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Строки, которые не имеют рерайта

In [14]:
groups1 = [group for group in groups if len(group) == 1]

In [15]:
len(groups1)

143

In [16]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том этим сейчас занимается.'],
 ['Том был просто не готов.'],
 ['Тому это тоже не нравится.'],
 ['Сколько сейчас времени в Париже?'],
 ['Тому было больше не к кому обратиться.'],
 ['Он сказал, что вчера был дома.'],
 ['Они никогда меня не слушают.'],
 ['Я поймал сегодня три рыбы.'],
 ['Я хочу что-нибудь сделать для Тома.'],
 ['Я сделал всё правильно?'],
 ['Я тоже завтра пойду в университет.'],
 ['Мне никогда не нужно было столько денег.'],
 ['Том не может жить без Мэри.'],
 ['Почему они так с ним поступают?'],
 ['Что могло с ними случиться?'],
 ['Мы будем делать всё по-своему.'],
 ['Где был Том весь день?'],
 ['Я здесь часто ем.'],
 ['Я всё ей рассказываю.'],
 [

### Сохранение результата

In [17]:
save_groups.save_groups(groups,"../output/solution1-result.json")